In [ ]:
from IPython import display
from IPython.core.display import Image

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Degree in Data Science and Engineering, group 96
## Machine Learning 2
### Fall 2023

&nbsp;
&nbsp;
&nbsp;
# Lab 2. Support Vector Machines for Classification

&nbsp;
&nbsp;
&nbsp;

**Emilio Parrado Hernández**

Dept. of Signal Processing and Communications

&nbsp;
&nbsp;
&nbsp;




<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />

# Loading the dataset

In this assignment you will work with an adaptation of the UCI repository dataset [segmentation]('https://archive.ics.uci.edu/ml/datasets/image+segmentation'). It is a binary classification task (labels are $+1$ and $-1$, respectively).

The following cell loads the data and constructs the corresponding training and testing sets.

In [ ]:
train_data = np.loadtxt('segmentation.data', delimiter=',')
X_train = train_data[:,1:]
y_train = train_data[:,0]
test_data = np.loadtxt('segmentation.test', delimiter=',')
X_test = test_data[:,1:]
y_test = test_data[:,0]

# 1.- Linear SVC

The first method we are going to use to solve the problem is a SVM endowed with a **linear kernel**. The performance of this model depends on the value of the regularization hyperparameter $C$.

## 1.1- Validation the hard way

In order to find which is the best value of $C$ in order to achieve the best generalization (a great performance on the test set without using the test set we are implementing the following **validation the hard way** procedure:

- Split **the training dataset** in two subsets:
    - actual training subset (70% of the data): `Xr` (observations) and `Yr` (targets)
    - validation subset (30% of the data): `Xv` and `Yv`

- Define `v_C` as a list with the following values for the parameter $C$: $10^{-3},$ $10^{-2},$ $10^{-1},$ $1,$ $10,$ $10^3,$ and $10^4$

 - Define `validation_accuracy` as an empty array with size equal to the length of `v_C`.
 
 - Run a loop with one iteration per member of `v_C`. At each iteration perform the following operations:
     - Instantiate a SVC with linear kernel and the value of $C$ corresponding to this iteration
     - Train the SVC with `Xr` and `Yr`
     - Evaluate the SVC with `Xv` and `Yv` and store the accuracy in the corresponding position of `validation_accuracy`
     
     
 

In [ ]:
#############
#           #
# YOUR CODE #
#           #
#############

In [ ]:
# do not edit this cell, just run it
plt.plot(np.array(v_C), validation_accuracy, label='validation')
plt.ylabel('accuracy')
plt.xlabel('C')
plt.grid()
plt.xscale('log')
_=plt.legend()

#### Discussion

- What is the value of $C$ that yields the better performance?

- What is the difference between the best value and the second best?

- Do you think it worths exploring larger or smaller values for $C$ (I mean, enlarging the explored range)?

#### your answer

Now we are going to check if the validation found the best parameter for the test set. In the next cell implement the following code:

- Define `test_accuracy` as an empty array with size equal to the length of `v_C`.
 
 - Run a loop with one iteration per member of `v_C`. At each iteration perform the following operations:
     - Instantiate a SVC with linear kernel and the value of $C$ corresponding to this iteration
     - Train the SVC with `X_train` and `y_train`
     - Evaluate the SVC with `X_test` and `y_test` and store the accuracy in the corresponding position of `test_accuracy`

In [ ]:
#############
#           #
# YOUR CODE #
#           #
#############

In [ ]:
# do not edit this cell, just run it
plt.plot(np.array(v_C), validation_accuracy, label='validation')
plt.plot(np.array(v_C), test_accuracy, label='test')
plt.ylabel('accuracy')
plt.xlabel('C')
plt.grid()
plt.xscale('log')
_=plt.legend()

#### Discussion

- What is the value of $C$ that would yield the better performance in the test set?

- What is the difference between the best accuracy found for the test set and the accuracy that you will obtain if you use the value of $C$ found with validation?

- Do you think it worths exploring larger or smaller values for $C$ (I mean, enlarging the explored range)?

#### your answer

## 1.2.- GridSearchCV

The next step is to use crossvalidation to do the hyperparameter search in a more systematic and clean way.

### Grids of hyperparameters

This method consists in to form a **grid** with a number of dimensions equal to the number of hyperparameters that one needs to optimize. The size of each dimension of the grid is equal to the number of values in the range of the corresponding hyperparameter. Notice that this method explores **discrete** ranges for each hyperparameter.

For the SVC with linear kernel we will initially explore the following range:


- `C` $ \in$ $[10^{-3},$ $10^{-2},$ $10^{-1},$ $1,$ $10,$ $10^3,$  $10^4]$

Notice this range determine a $7 \times 1$ grid. 

In models that depend on a larger number of hyperparameters one has to be careful with the granularity of the ranges as the combinatorial explosion of the size of the grid can be hard to manage.

### Cross validation 

Cross validation is a commonly used procedure in machine learning to simulate the effect of training a model with a set of data and evaluate its generalization capabilities as the performance in a **separate dataset**. 

The cross validation process involves the following steps:

- Randomly partition the training dataset in $N$ disjoint subsets of similar sizes. Each of this subsets is called **fold** in machine learning jargon. Hence the term **N-fold cross validation**.

- Let us suppose we have chosen $N=3$ folds. This means the training data has been split in three subsets: $(X_1, Y_1)$, $(X_2, Y_2)$ y $(X_3, Y_3)$. 

- Create an instance of the model with the corresponding hyperparameters. The cross validation follows with the execution of the following loop

    For $n=1,2,\dots,N$ iterations:  
    1. Choose $(X_n,Y_n)$ as **validation set** for iteration $n$
    2. Prepare a **training set** for iteration $n$ joining the rest of the subsets (excluding the validation set)
    3. Fit the model instance with the training set of step 2
    4. Evaluate  the model instance (method `score`) with the validation set of step 1
    5. Keep the *score* achieved in the $n$ iteration.

- Once the loop is finished, we have $N$ scores, each corresponding to the evaluation of the model fitted in each iteration with the corresponding validation set.
- Estimate the **real score** that an instance of the model fitted using all the data would yield in a separate dataset computing the **mean** and **standard deviation** of the $N$ validation scores.

Typical values for the number of folds include $N\in \{3, 5, 10\}$


###  Cross validation to explore the grid

The grid is explored by a loop that visits all its nodes and runs a  **cross validation** to estimate the test performance that the model would yield if it were fitted using the values for hyperparameter that correspond to that node. 

Once all the nodes of the grid have been cross validated, the procedure outputs the combination of hyperparameters corresponding to the node with the best cross validation performance. 

###  Grid search in sklearn

There is a module in sklearn that implements this algorithm for exploring a grid of hyperparameters with cross validation: [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV)


In the following cell implement code that:
- define `param_lsvc`, a dictionary that will serve as dictionary of parameters for the range of $C$
- instantiate `grid_lsvc`, an `GridSearchCV` object with a SVC with linear kernel as estimator and `param_lsvc` as parameter dictionary.
- Train `grid_lsvc` with `X_train` and `y_train`

In [ ]:
#############
#           #
# YOUR CODE #
#           #
#############

Now look at attribute `grid_lsvc.cv_results_` to identify fields that store:
- the values of parameter $C$ used in each node of the grid. Store them in an array called `values_of_C`
- the average score achieved by the model in each node of the grid. Store them in an array called `Mean_acc`
- the standard deviation of the scores achieved by the model in each node of the grid. Store them in an array called `Std_acc`

In [ ]:
#############
#           #
# YOUR CODE #
#           #
#############

In [ ]:
# do not edit this cell, just run it
plt.plot(values_of_C, Mean_acc, label='mean validation', color='blue')
plt.fill_between(values_of_C, 
                 Mean_acc - Std_acc, 
                 Mean_acc + Std_acc,
                 label='std validation',
                 alpha=0.2, 
                 color='blue')
plt.plot(values_of_C, test_accuracy, label='test', color='green')
plt.ylabel('accuracy')
plt.xlabel('C')
plt.grid()
plt.xscale('log')
_=plt.legend()

#### Discussion

- What is the value of $C$ that suggest GridSearchCV?

- What is the difference between the best accuracy found for the test set and the accuracy that you will obtain if you use the value of $C$ found by GridSearchCV?

- Is there any significant difference between GridSearchCV and validation the hard way?

#### your answer

## 1.3.- Running times

GridSearchCV also provides with information about the times needed to train and to evaluate the algorithm. Identify what are the fields that provide with that information inside `grid_lsvc.cv_results_` and produce two plots, similar to the previous one in which you show:
- plot 1: The time (in seconds) employed to train the model vs. the value of $C$
- plot 2: The time (in seconds) employed to evaluate the model vs. the value of $C$

In [ ]:
#############
#           #
# YOUR CODE #
#           #
#############

#### Discussion

- How is the evolution of the training time with $C$?

- How is the evolution of the evaluation time with $C$?

#### your answer

# 2.- Explore different kernels

After obtaining results with the linear kernel, we will explore the performance of the SVM endowed with RBF and polynomial kernels.

## 2.1.- RBF kernel

The RBF kernel introduces a new hyperparameter in the model, the spread of the kernel $\gamma$. We will consider the following range of values: 
$$
\gamma \in [10^{-5}, 10^{-4}, 10^{-3}, 10^{-2}, 10^{-1}, 1, 10]
$$

Therefore our crossvalidation grid search needs to expand one more dimension to allocate this range. 

In the next cell implement code that:
- Defines `v_g`, an array with the range of values for `gamma`
- Defines `param_rbf_svc`, a dictionary with the ranges of $\gamma$ and $C$ for the SVM
- Creates `grid_rbf_svc`, a `GridSearchCV` object with an SVC as estimator and  `param_rbf_svc` as parameter dictionary
- Train `grid_rbf_svc` with `X_train` and `y_train`

In [ ]:
#############
#           #
# YOUR CODE #
#           #
#############

Now we need to retrieve useful information from attribute `cv_results_`, in particular:
- store in `C_values` the values of parameter $C$ used in each node of the grid
- store in `gamma_values` the values of parameter $\gamma$ used in each node of the grid
- store in `Acc` the average validation score obtained in each node of the grid


In [ ]:
#############
#           #
# YOUR CODE #
#           #
#############

In [ ]:
# do not edit this cell, just run it
results_rbf = (pd.DataFrame(np.vstack((C_values, 
                                 gamma_values,
                                 Acc)).T,
                      columns=['C','gamma','Acc'])).pivot(index='C', columns='gamma')


results_rbf

In [ ]:
plt.figure()
for iC, C in enumerate(results_rbf.index):
    plt.plot(v_g, results_rbf.loc[C].values, label='C={0:f}'.format(C))
plt.xscale('log')
plt.grid()
_=plt.legend()

#### Discussion

- What are the best values of $C$ and $\gamma$ that yield the better performance in crossvalidation?

- Do you think it worths exploring larger or smaller values for $C$ and/or $\gamma$?

#### your answer

The GridSearchCV object has an attribute that facilitates the identification of the best set of hyperparameters (`best_params_`) and also methods to consume the model trained with these best hyperparameters: `score()` and `predict()`.
Print the accuracy in the test set achieved by the SVC with RBF kernel and hyperparameters found by GridSearchCV

In [ ]:
#############
#           #
# YOUR CODE #
#           #
#############

#### Discussion

- What kernel gives the best performance in the test set, after tuning the hyperparameters using crossvalidation?



#### your answer

## 2.2.- Polynomial kernel

The polynomial kernel introduces a new hyperparameter in the model, the degree of the polynomial $d$. We will consider the following range of values: 
$$
d \in [2,3,4,5,6,7,8,9,10]
$$

Therefore our crossvalidation grid search needs to allocate this new range. 

In the next cell implement code that:
- Defines `v_d`, an array with the range of values for $d$
- Defines `param_poly_svc`, a dictionary with the ranges of $d$ and $C$ for the SVM
- Creates `grid_poly_svc`, a `GridSearchCV` object with an SVC as estimator and  `param_poly_svc` as parameter dictionary
- Train `grid_poly_svc` with `X_train` and `y_train`

In [ ]:
#############
#           #
# YOUR CODE #
#           #
#############

Now we need to retrieve useful information from attribute `cv_results_`, in particular:
- store in `C_values_poly` the values of parameter $C$ used in each node of the grid
- store in `d_values` the values of parameter $d$ used in each node of the grid
- store in `Acc_poly` the average validation score obtained in each node of the grid


In [ ]:
#############
#           #
# YOUR CODE #
#           #
#############

In [ ]:
# do not edit this cell, just run it
results_poly = (pd.DataFrame(np.vstack((C_values_poly, 
                                 d_values,
                                 Acc_poly)).T,
                      columns=['C','d','Acc'])).pivot(columns='d', index='C')
results_poly

In [ ]:
plt.figure()
for C in results_poly.index:
    plt.plot(v_d, results_poly.loc[C].values, label='C={0:f}'.format(C))

plt.grid()
_=plt.legend()

#### Discussion

- What are the best values of $C$ and $d$ that yield the better performance in crossvalidation?

- Do you think it worths exploring larger or smaller values for $C$ and/or $d$?

- What is more critial for the performance of the classifier, the selection of $C$ or the degree?

- What is the performance of the SVM with polynomial kernel and parameters tuned using crossvalidation in the test set? How does it compare with the other two kernels?

#### your answer

# 3.- Scaling

The last step in the development of the model involves a data preprocessing with a Standard Scaler. Since the scaling can affect the performance achieved by the SVC endowed by any of the three kernels, we will carry out a single validation procedure that searches the best option within the three kernels.

For this purpose, in the next cell write code that:
- defines `pipe_svc`, a `Pipeline` with a first stage that is an `StandardScaler` followed by a SVC
- defines `param_svc` as the object needed to pass the ranges of hyperparameters to the SVC inside the Pipeline. Check the documentation of GridSearchCV and Pipeline. In essence, `param_svc` needs to be a list of three dictionaries, each dictionary with the ranges of parameters needed by each kernel.
- Create `grid_sc_svc`, the `GridSearchCV` object with `pipe_svc` as estimator and `param_svc` as parameter dictionary
- Train `grid_sc_svc` with `X_train` and `y_train`

In [ ]:
#############
#           #
# YOUR CODE #
#           #
#############

#### Discussion

- What is the configuration of the best classifier found by crossvalidation?

- What is the impact of the scaling in the performance of the classifier?
